In [ ]:
'''
    Project - Quantum Key Distribution implementation.
    Author - Rezi Gelenidze
    Date - 03.01.22
    
    * run all blocks to create and see eavesdropper scenario.
    
    * run all blocks except "Phase 2" to create scenario
    when key is distributed without eavesdropper interception.
'''

# import necessary modules
import random
from qiskit import QuantumCircuit, execute, Aer

In [ ]:
# predefined functions
def encode_qubits(bits, bases):
    ''' 
        bits - list of bits in str type ('0' or '1')
        bases - list of bases ('X' or 'Z')
        
        job - encodes new qubits by scheme with given bits and bases
        
        predefined scheme:
            Z   X
        0  |0> |+>
        1  |1> |->
        
        return - list of encoded qubits
        
    '''
    
    # ensure that same amount of bits and bases are recieved
    assert len(bits) == len(bases)
    
    encoded_qubits = []
    for bit, base in zip(bits, bases):
        # create new qubit by the encoding scheme
        new_qc = QuantumCircuit(1, 1)
       
        # check states and bits to decide how to manipulate
        if bit == '0':
            if base == 'Z':
                # 0 - Z -> |0>
                pass
            elif base == 'X':
                # 0 - X -> |+>
                new_qc.h(0)
        elif bit == '1':
            if base == 'Z':
                # 1 - Z -> |1>
                new_qc.x(0)
            elif base == 'X':
                # 1 - X -> |->
                new_qc.x(0)
                new_qc.h(0)
        
        encoded_qubits.append(new_qc)
        
    return encoded_qubits


def measure_qubits(qubits, bases):
    '''
        qubits - recieved list of qubits to measure
        bases - list of bases ('X' or 'Z') to measure qubits in
        
        job - Measures qubits with given bases
        
        return - list of bits given from performed measurement
    '''
    result_bits = []
    
    for qubit, base in zip(qubits, bases):
        # add measurements to qubits with given base
        if base == 'Z':
            qubit.measure(0,0)
        elif base == 'X':
            qubit.h(0)
            qubit.measure(0,0)
            
        # run measurements
        job = execute(qubit, backend=Aer.get_backend('qasm_simulator'), shots = 1) 
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)
        
        result_bits.append(measured_bit)
        
    return result_bits


def generate_bits(n):
    ''' n amount of bits generator '''
    return [random.choice(['0', '1']) for i in range(n)]


def generate_bases(n):
    ''' n amount of bases generator '''
    return [random.choice(['Z', 'X']) for i in range(n)]    


def eliminate_differences(bits, indexes):
    ''' 
        new list of bits is created by appending bits from user bits,
        where base indexes are matched. Used in comparing phase to filter out
        agreed bits by matched bases
    '''
    result_key = []
    for i in range(len(bits)):
        if i in indexes:
            result_key.append(bits[i])
    
    return result_key

In [ ]:
''' Phase 1: Generate, encode and send '''

# Alice generates 500 random bits and bases
alice_bits = generate_bits(500)
alice_bases = generate_bases(500)

# Encoding qubits by generated bits and bases
encoded_qubits = encode_qubits(alice_bits, alice_bases)

# print data 
print('Alice Bits:')
print(alice_bits)

print('Alice Bases:')
print(alice_bases)

# qubits are sent to bob with quantum channel ...

In [ ]:
''' 
    Phase 2: Eavesdropper (Eve) intercepts alice's qubits on quantum channel.
    Eve measures qubits and sends them to Bob to remain undetected.
'''

# measure all qubits with random bases and save result bits in a list
eve_bases = generate_bases(500)
eve_bits = measure_qubits(encoded_qubits, eve_bases) # code in 'predefined functions'

# print data 
print('Eve Bases:')
print(eve_bases)

print('Eve Bits:')
print(eve_bits)

# After measurement eve sends qubits to Bob to remain undetected...

In [ ]:
''' 
    Phase 3: Bob recieves qubits from Alice, bob does 
    not know yet that qubits are intercepted by Eve
'''

# bob measures recieved qubits with random bases
bob_bases = generate_bases(500)
bob_bits = measure_qubits(encoded_qubits, bob_bases)

# print data 
print('Bob Bases:')
print(bob_bases)

print('Bob Bits:')
print(bob_bits)

In [ ]:
''' Phase 4: Comparing '''

# alice and bob compare bases on public channel, both eliminates own bits where bases are different.

# determine indexes of matched bases in a list
same_base_indexes = []
for i in range(len(alice_bases)):
    if alice_bases[i] == bob_bases[i]:
        same_base_indexes.append(i)
        
# Alice generates own key by eliminating bits
# encoding with different bases by Alice and Bob
alice_key = eliminate_differences(
    alice_bits,
    same_base_indexes
)

print('Alice key:')
print(alice_key)

# Bob does the same...
bob_key = eliminate_differences(
    bob_bits,
    same_base_indexes
)

print('Bob key:')
print(bob_key)


# if keys are different, qubits are surely intercepted by someone.
# else, secret key is safe to encrypt information with
if alice_key == bob_key:
    print('Key is safe to use!')
    print(f'Key length: {len(alice_key)}') 
else:
    print('\n Key is compromised and is not safe!')

In [ ]:
# OPTIONAL - using key to send information (scenario when key is not compromise)
import binascii

def encrypt_message(unencrypted_string, key):
    # Convert ascii string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # created the encrypted string using the key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # created the unencrypted string using the key
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Convert bitstring into
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

In [ ]:
# Alice encrypts secret message with key that is already distributed.
secret_message = 'Quantum Computing is cool :)'
encrypted_message = encrypt_message(secret_message, alice_key)

print('Alice message:', secret_message)
print('\nEncrypted message:', encrypted_message)


# Alice sends encrypted message to Bob on public channel ...

In [ ]:
# Bob recieved encrypted message and decrypts it with own key.
decrypted_message = decrypt_message(encrypted_message, bob_key)

print('Message decrypted by Bob:', decrypted_message)